# Proximal Policy Optimization

---
Code based on Udacity exercise on PPO to solve the Atari-pong game using the pixels of two succeeding frames as an input.

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
#env = UnityEnvironment(file_name='C:\EigeneLokaleDaten\DeepRL\Value-based-methods\p2_continuous-control\Reacher_Windows_x86_64_single_agent\Reacher.exe')
env = UnityEnvironment(file_name='C:\EigeneLokaleDaten\DeepRL\Value-based-methods\p2_continuous-control\Reacher_Windows_x86_64_multiple_agents\Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


## Train a PPO agent - use classroom exercise code to start exploring hyperparameter and network architecture

In [109]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device: ",device)

# for the moment start with one agent alone - no paralell code execution 
#import PPO_utils
#device = PPO_utils.device
#print("using device: ",device)

using device:  cpu


In [110]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)


# set up a fully connected (fc) neural net
# the output is the action vector (dim=4)
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.state_dim = 33
        self.action_dim = 4
        self.fc1 = nn.Linear(self.state_dim, 64*4)
        self.fc2 = nn.Linear(64*4, 32*4)
        self.fc3 = nn.Linear(32*4, 16*2)
        self.fc4 = nn.Linear(16*2,self.action_dim)        
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc4.weight.data.uniform_(-3e-3, 3e-3)
        
    def forward(self, x):  
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        # flatten the tensor - do we have to do this? 
        # probably not because no CNN 
        ## x = x.view(-1,self.size2)
        return torch.tanh(self.fc4(x))

# run your own policy!
policy=Policy().to(device)

# we use the adam optimizer with learning rate 2e-4
# optim.SGD is also possible
import torch.optim as optim
optimizer = optim.Adam(policy.parameters(), lr=1e-3)

In [111]:
policy=Policy().to(device)
env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
#actions = policy(torch.from_numpy(np.array(states)[0]).float().to(device)).squeeze().cpu().detach().numpy()      
print(len(states))
print(states[0])
actions = policy(torch.from_numpy(states[0]).float().to(device)).squeeze().cpu().detach().numpy()      
print(actions)

20
[ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -5.35304642e+00 -1.00000000e+00
 -5.94515991e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.90005660e-01]
[-0.02160235  0.14855763  0.17048088  0.08612862]


In [112]:
do_scaling = True
if do_scaling:
    print('load state scaling')
    data = np.load('state_scale.npz')
    scale = data['scale_int']
else:
    scale = np.ones(33,)

# collect trajectory for a single agent
def collect_trajectory(env, policy, tmax=200):    
    debug_ = False 
    num_agents = 20
    action_size = 4
    
    #initialize returning lists and start the game!
    state_list=[]
    reward_list=[]
    prob_list=[]
    actions_list=[]

    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    states = states/scale
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    
    for jj in range(1):
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        # print(actions.shape)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment

    if debug_:
        print('ACTION:',actions)
    next_states = env_info.vector_observations         # get next state (for each agent)   
    next_states = next_states/scale

    if debug_:
        print('START STATES:')
        print(states)
        print(type(states))
        print(states.shape)
        #print(np.array(states)[0])
        
    actions = policy(torch.from_numpy(states).float().to(device)).squeeze().cpu().detach().numpy()
                     
    #actions = policy(torch.from_numpy(states).float().to(device)).squeeze().cpu().detach().numpy()        
    if debug_:
        print('###')
        print('ACTION after Policy:',actions)
        print('###')
                
    for t in range(tmax):   
        # action will only be used as the pi_old
        # no gradient propagation is needed
        # so we move it to the cpu
        actions = policy(torch.from_numpy(states).float().to(device)).squeeze().cpu().detach().numpy()        
        probs = actions
        
        # advance the game 
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        next_states = next_states/scale
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step        
        
        # store the result
        prob_list.append(probs)
        state_list.append(torch.from_numpy(states).float().to(device))
        reward_list.append(rewards)
        actions_list.append(actions)
        
        # stop if any of the trajectories is done
        # we want all the lists to be retangular
        if np.any(dones):                                  # exit loop if episode finished
            break


    # return pi_theta, states, actions, rewards, probability
    return prob_list, state_list, \
        actions_list, reward_list

load state scaling


In [113]:
actions = np.random.randn(num_agents, action_size)
print(actions.shape)
actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
print(actions.shape)
env_info = env.step(actions)[brain_name]           # send all actions to tne environment

(20, 4)
(20, 4)


In [114]:
for jj in range(1):  # two sample trajectories...
    prob_list, state_list, \
        actions_list, reward_list = collect_trajectory(env,policy,tmax=3)
    print(reward_list)
    print(actions_list)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[array([[-0.02086769,  0.1479469 ,  0.17112812,  0.08634928],
       [-0.02081916,  0.14807954,  0.17106716,  0.08638076],
       [-0.02083855,  0.1479851 ,  0.17115302,  0.08636311],
       [-0.02087497,  0.1479163 ,  0.17116608,  0.08631735],
       [-0.02087273,  0.14787911,  0.17121433,  0.08637455],
       [-0.02085084,  0.14803569,  0.17116156,  0.0864339 ],
       [-0.02083923,  0.14792182,  0.17111334,  0.08630627],
       [-0.02087435,  0.14787714,  0.17121416,  0.08637424],
       [-0.02086331,  0.14793684,  0.17120375,  0.08639609],
       [-0.02084338,  0.14807245,  0.17109151,  0.08639788],
       [-0.02086153,  0.14793916,  0.17119616,  0.08633665],
       [-0.02082309, 

In [115]:
# clipped surrogate function
# similar as -policy_loss for REINFORCE, but for PPO
def clipped_surrogate(policy, old_probs, states, actions, rewards,
                      discount=0.995,
                      epsilon=0.1, beta=0.01, debug_=False):
    
    discount = discount**np.arange(len(rewards))
    rewards = np.asarray(rewards)*discount[:,np.newaxis]
    
    # convert rewards to future rewards
    rewards_future = rewards[::-1].cumsum(axis=0)[::-1]
    
    mean = np.mean(rewards_future, axis=1)
    std = np.std(rewards_future, axis=1) + 1.0e-10

    rewards_normalized = (rewards_future - mean[:,np.newaxis])/std[:,np.newaxis]
    
    if debug_:
        print('actions:',actions)
    # convert everything into pytorch tensors and move to gpu if available
    actions = torch.tensor(actions, dtype=torch.float, device=device)  # this was int8 before :( 
    old_probs = torch.tensor(old_probs, dtype=torch.float, device=device)
    rewards = torch.tensor(rewards_normalized, dtype=torch.float, device=device)
    
    if debug_:
        print('### discounted rewards',rewards)
        print('### discounted future rewards',rewards_future)
        print('mean:',mean)
        print('std:',std)
        print('### discounted future rewards normalized ',rewards_normalized)
        rewards = torch.tensor(rewards_normalized.copy(), dtype=torch.float, device=device)
        print('actions:',actions)

    # convert states to policy (or probability)
    new_probs = states_to_prob(policy, states)
    #new_probs = torch.where(actions == RIGHT, new_probs, 1.0-new_probs)
    
    # ratio for clipping
    ratio = new_probs/old_probs
            
    # clipped function
    clip = torch.clamp(ratio, 1-epsilon, 1+epsilon)

    rewards = rewards[:,:,None]
    clipped_surrogate = torch.min(ratio*rewards, clip*rewards)
    
    if debug_:
        print('RATIO: Shape:',ratio.shape)
        print('REWARDS: Shape:',rewards.shape)
        print('ratio',ratio)
        print('clip',clip)
        print('rewards',rewards)
        print('clipped surrogate',clipped_surrogate)

    
    if debug_:
        print('ratio*rewards',(ratio*rewards).shape)
        print('clip*rewards',(clip*rewards).shape)
    


    if debug_:
        print('new_probs',new_probs)
        print('old_probs',old_probs)

    # include a regularization term
    # this steers new_policy towards 0.5
    # add in 1.e-10 to avoid log(0) which gives nan
    '''
     entropy = -(new_probs*torch.log(old_probs+1.e-10)+ \
        (1.0-new_probs)*torch.log(1.0-old_probs+1.e-10))
    '''            

    # entropy = -(np.abs(new_probs.detach().numpy())*np.log(np.abs(old_probs.detach().numpy())+1.e-10)+ \
    #    (1.0-np.abs(new_probs.detach().numpy()))*np.log(1.0-np.abs(old_probs.detach().numpy())+1.e-10))
    
        
    if debug_:
        entropy = -(new_probs*torch.log(old_probs+1.e-10)+ \
        (1.0-new_probs)*torch.log(1.0-old_probs+1.e-10))
        print(entropy.shape)
        print('entropy',entropy)
    
    
    # this returns an average of all the entries of the tensor
    # effective computing L_sur^clip / T
    # averaged over time-step and number of trajectories
    # this is desirable because we have normalized our rewards
    #return torch.mean(clipped_surrogate + beta*entropy)
    return torch.mean(clipped_surrogate)

In [116]:
# convert states to probability, passing through the policy
'''
def states_to_prob(policy, states):
    # states = torch.stack(states)
    # policy_input = states.view(-1,*states.shape[-3:])
    # return policy(policy_input).view(states.shape[:-3])
    return policy(torch.from_numpy(np.array(states)).float().to(device))
'''

# convert states to probability, passing through the policy
def states_to_prob(policy, states):
    states = torch.stack(states)
    #policy_input = states.view(-1,*states.shape[-3:])
    #return policy(policy_input).view(states.shape[:-3])
    return(policy(states))


In [117]:
print('debug..')
prob_list, state_list, \
        actions_list, reward_list = collect_trajectory(env,policy,tmax=1)
old_probs = prob_list
rewards = reward_list
states = state_list
actions = actions_list
print(len(rewards))
rewards[0][3] += 1  # hack some reward value != 0 to test surrogate function 
rewards[0][8] += 1  # hack some reward value != 0 to test surrogate function 
clipped_surrogate(policy, old_probs, states, actions, rewards, epsilon=0.2, beta=0.1, debug_=True)

debug..
1
actions: [array([[-0.02087181,  0.14810827,  0.17109473,  0.08639201],
       [-0.02085977,  0.14794512,  0.17120185,  0.08636135],
       [-0.02083842,  0.14810878,  0.1710728 ,  0.0863828 ],
       [-0.02086454,  0.14800112,  0.17112015,  0.08637721],
       [-0.02084027,  0.1479171 ,  0.1711317 ,  0.08629676],
       [-0.02085538,  0.14810692,  0.17108941,  0.08639178],
       [-0.02088744,  0.1478644 ,  0.17121883,  0.08636571],
       [-0.02086202,  0.1479185 ,  0.17120484,  0.0863919 ],
       [-0.02086016,  0.14796266,  0.17120051,  0.08636756],
       [-0.02085374,  0.14811128,  0.17108425,  0.08638857],
       [-0.02087888,  0.14811729,  0.1711098 ,  0.08640037],
       [-0.02085363,  0.14803852,  0.17109661,  0.08638043],
       [-0.02088565,  0.14787242,  0.17121148,  0.08635979],
       [-0.0208938 ,  0.14786276,  0.17122014,  0.0863592 ],
       [-0.02079847,  0.14799997,  0.17106675,  0.08635398],
       [-0.02085214,  0.14802736,  0.17113166,  0.08640882],
    

tensor(-8.5194e-05, grad_fn=<MeanBackward0>)

In [118]:
actions = np.random.randn(num_agents, action_size)
#print(actions.shape)
actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
print(actions.shape)
env_info = env.step(actions)[brain_name]           # send all actions to tne environment
prob_list, state_list, \
        actions_list, reward_list = collect_trajectory(env,policy,tmax=1)
print(len(state_list))
print(state_list[0].shape)
states_to_prob(policy,state_list)

(20, 4)
1
torch.Size([20, 33])


tensor([[[-0.0209,  0.1479,  0.1711,  0.0863],
         [-0.0209,  0.1480,  0.1712,  0.0864],
         [-0.0209,  0.1481,  0.1711,  0.0864],
         [-0.0209,  0.1480,  0.1712,  0.0864],
         [-0.0208,  0.1480,  0.1712,  0.0865],
         [-0.0208,  0.1480,  0.1711,  0.0864],
         [-0.0209,  0.1481,  0.1711,  0.0864],
         [-0.0209,  0.1479,  0.1713,  0.0864],
         [-0.0208,  0.1479,  0.1712,  0.0864],
         [-0.0209,  0.1481,  0.1711,  0.0864],
         [-0.0208,  0.1480,  0.1711,  0.0864],
         [-0.0209,  0.1480,  0.1712,  0.0864],
         [-0.0209,  0.1480,  0.1712,  0.0864],
         [-0.0209,  0.1480,  0.1712,  0.0864],
         [-0.0209,  0.1481,  0.1711,  0.0864],
         [-0.0209,  0.1479,  0.1712,  0.0864],
         [-0.0208,  0.1481,  0.1710,  0.0864],
         [-0.0209,  0.1479,  0.1712,  0.0863],
         [-0.0208,  0.1480,  0.1711,  0.0863],
         [-0.0209,  0.1481,  0.1711,  0.0864]]], grad_fn=<TanhBackward0>)

In [119]:
#!pip install progressbar
#import progressbar as pb
from collections import deque

In [121]:
## from parallelEnv import parallelEnv
import numpy as np

# training loop max iterations
episode = 10000

scores_deque = deque(maxlen=100)

'''
# widget bar to display progress
!pip install progressbar
import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episode).start()
'''

##envs = parallelEnv('PongDeterministic-v4', n=8, seed=1234)

# hyperparameter to set 
discount_rate = .99999
epsilon = 0.1
epsilon_decay = 1
beta = .05
beta_decay = 1
#tmax = 320
tmax = 990
SGD_epoch = 4

# keep track of progress
mean_rewards = []

for e in range(episode):

    # collect trajectories
    #old_probs, states, actions, rewards = \
    #    pong_utils.collect_trajectories(envs, policy, tmax=tmax)
    
    old_probs, states, actions, rewards = \
        collect_trajectory(env, policy, tmax=tmax)
                
    total_rewards = np.sum(rewards, axis=0)

    # gradient ascent step
    for _ in range(SGD_epoch):
        L = -clipped_surrogate(policy, old_probs, states, actions, rewards, epsilon=epsilon, beta=beta)
        optimizer.zero_grad()
        L.backward()
        #torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)   # take advise from Attempt 3 / Udacity Course 
        torch.nn.utils.clip_grad_norm(policy.parameters(), 1)   # take advise from Attempt 3 / Udacity Course 
        optimizer.step()
        del L
    
    # the clipping parameter reduces as time goes on
    epsilon*=epsilon_decay
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta*=beta_decay
    
    # get the average reward of the parallel environments
    #mean_rewards.append(np.mean(total_rewards))
    
    scores_deque.append(np.mean(total_rewards))
    #scores.append(np.mean(total_rewards))
    print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(e, np.mean(scores_deque), np.mean(total_rewards)), end="")
    #print("Episode: {0:d}, score: {1:f}, score: {1:f}".format(e+1,np.mean(total_rewards),np.mean(mean_rewards)))
    # display some progress every 20 iterations
    if (e+0)%500 ==0 :
        #print("Episode: {0:d}, score: {1:f}".format(e+1,np.mean(mean_rewards)))
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(e, np.mean(scores_deque), np.mean(total_rewards)))
        #print(total_rewards)

        
'''        
    # update progress widget bar
    timer.update(e+1)
    
timer.finish()
'''

C:\ProgramData\Anaconda3\envs\drlnd\lib\site-packages\ipykernel_launcher.py:50: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Episode 0	Average Score: 0.59	Score: 0.59
Episode 500	Average Score: 0.50	Score: 0.46
Episode 1000	Average Score: 0.49	Score: 0.60
Episode 1500	Average Score: 0.49	Score: 0.60
Episode 2000	Average Score: 0.50	Score: 0.56
Episode 2500	Average Score: 0.49	Score: 0.54
Episode 3000	Average Score: 0.50	Score: 0.71
Episode 3500	Average Score: 0.49	Score: 0.40
Episode 4000	Average Score: 0.48	Score: 0.50
Episode 4500	Average Score: 0.48	Score: 0.55
Episode 5000	Average Score: 0.49	Score: 0.54
Episode 5500	Average Score: 0.50	Score: 0.49
Episode 6000	Average Score: 0.49	Score: 0.61
Episode 6151	Average Score: 0.50	Score: 0.28

KeyboardInterrupt: 

In [ ]:
e
np.mean(mean_rewards)
import matplotlib.pyplot as plt
plt.figure()
plt.plot(mean_rewards)

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
#actions = policy(torch.from_numpy(np.array(states)[0]).float().to(device)).squeeze().cpu().detach().numpy()      
print(states[0])
actions = policy(torch.from_numpy(states[0]).float().to(device)).squeeze().cpu().detach().numpy()      
print(actions)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```